#Cleaning data

Campaign finance information for Colorado is publicly available from the [TRACER](http://tracer.sos.colorado.gov/PublicSite/homepage.aspx),Transparency in Contribution and Expenditure Reporting, website.  You may also use the REST [API](http://co.opencampaigndata.org/).


In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv(os.path.join('data','exp_sub.csv'))
print df.head()

             filedDate firstName                           lastName  \
0  2013-03-13 00:00:00       NaN                           KEY BANK   
1  2013-03-13 00:00:00       NaN                           KEY BANK   
2  2013-01-11 00:00:00       NaN                     CITI BANK VISA   
3  2013-04-15 00:00:00       NaN                            NGP VAN   
4  2013-04-14 00:00:00       NaN  LEADERSHIP PROGRAM OF THE ROCKIES   

                             address        city state    zip  amount  
0                       PO BOX 22114       ABANY    NY  12201     NaN  
1                       PO BOX 22114       ABANY    NY  12201     NaN  
2                                . .           .    CO  .....     NaN  
3              48 GROVE ST SUITE 202  SOMERVILLE    MA  02144     NaN  
4  1777 S. HARRISON STREET, STE. 807      DENVER    CO  80210     NaN  


In [3]:
print df['zip'].unique()

['12201' '.....' '02144' ..., '32696.0' '40401.0' '80807-']


In [4]:
print df['state'].unique()

['NY' 'CO' 'MA' 'ID' 'CA' 'MN' 'NC' 'GA' 'WA' 'DC' 'NJ' 'TX' 'KY' 'TN' 'IN'
 nan 'AZ' 'MD' 'UT' 'PA' 'WI' 'FL' 'OH' 'NE' 'IL' 'D.' 'WV' 'DE' 'OT' 'MS'
 'OK' 'MI' 'DI' 'LA' 'VA' 'ARVADA' 'BC' 'NV' 'TE' 'IA' 'MO' 'N/' 'NM' 'SD'
 'ND' 'HI' 'ON' 'NH' 'CT' 'AL' 'KS' 'OR' 'QC' 'AR' 'NB' 'VT' 'MY' 'LU' 'SC'
 'SO' 'WY' 'C0' 'UN' 'UK' 'MT' 'HA' 'CP' 'ME' 'BE' 'VI' 'IO' 'KE' 'CHECKS'
 'NO' 'PE' 'CANDIDATE NAME TAG' 'GAS & PARKING' 'POSTAGE' 'GAS' 'LO'
 'INK CARTRIDGE' 'T-SHIRTS'
 'ELECTRONIC DONATION PROCESSING AND TRANSFER FEES' 'PARADE FEE'
 'LARIMER DEMS BREAKFAST' 'TRXF FEES' 'PROPANE' 'FUEL' 'LODGING' 'AA'
 'MAILING SUPPLIES' 'PARK RESERVATION FEE' 'EU' 'KA' 'LITERATURE'
 'ICE CREAM FOR CAMPAIGN SOCIAL' 'GORA GORA ORKESTAR' 'BANK TRANSFER FEES'
 'XX' 'CANDIDATE CAMPAIGN CONTRIBUTION' 'TRUMAN CAMPAIGN DINNER' 'IR' 'AB'
 'POLITICAL YARD SIGNS' 'ROOM DIVIDER' 'POSTCARD PUBLICATION AND MAILING'
 'BANK TRXF FEES' 'GE' 'RI' 'PR' 'AK' 'AP' 'NS' 'BR' 'R.' 'DU' 'BO'
 'IN KIND CONTRIBUTION FROM CANDI

##Approach: use `map`

In [5]:
def parse_filed_date(dstr):
    try:
        tmp = str(dstr).split('-')
        year, month, day = tmp[0], tmp[1], tmp[2][:2]
        return datetime.date(year=int(year), month=int(month), day=int(day))
    except ValueError:
        return datetime.date(year=1990, month=1, day=1)
    except IndexError:
        return datetime.date(year=1990, month=1, day=1)

In [7]:
def cleanup_zip(zipcode):
    tmp = str(zipcode)
    if tmp.find('-') > 0:
        tmp = tmp[:tmp.find('-')]
    try:    
        tmp = str(abs(int(float(tmp))))
    except ValueError:
        tmp = '99999'
        
    if len(tmp) < 5:
        tmp = '99999'
    elif len(tmp) > 5:
        tmp = tmp[:5]
        
    return tmp

In [8]:
df['zip_clean'] = df['zip'].map(cleanup_zip)

In [9]:
df['date_clean'] = df['filedDate'].map(parse_filed_date)

In [10]:
print df.head(10)

             filedDate firstName                           lastName  \
0  2013-03-13 00:00:00       NaN                           KEY BANK   
1  2013-03-13 00:00:00       NaN                           KEY BANK   
2  2013-01-11 00:00:00       NaN                     CITI BANK VISA   
3  2013-04-15 00:00:00       NaN                            NGP VAN   
4  2013-04-14 00:00:00       NaN  LEADERSHIP PROGRAM OF THE ROCKIES   
5  2013-04-14 00:00:00       NaN                   CONSTANT CONTACT   
6  2013-04-15 00:00:00       NaN                           1ST BANK   
7  2013-07-15 00:00:00       NaN                           1ST BANK   
8  2013-07-15 00:00:00       NaN                           1ST BANK   
9  2013-01-09 00:00:00       NaN                        WELLS FARGO   

                             address               city state    zip  amount  \
0                       PO BOX 22114              ABANY    NY  12201     NaN   
1                       PO BOX 22114              ABANY   

What percent is bad?

In [11]:
print np.sum(df['zip_clean'] == '99999')/float(len(df))*100

7.89467765404


In [12]:
print np.sum(df['date_clean'] == datetime.date(year=1990, month=1, day=1))/float(len(df))*100

0.019723285761


Is that really a big deal?

In [13]:
dirty_amount = df[df['zip_clean'] == '99999']['amount'].sum()

In [14]:
clean_amount = df[df['zip_clean'] != '99999']['amount'].sum()

In [15]:
print dirty_amount/(clean_amount+dirty_amount)*100

1.93791959238


Less of an issue in terms of money.

## Do our results make sense?

In [16]:
df[df['zip_clean'] == '99999']['zip'].unique()

array(['.....', '02144', '02451', '08542', nan, '02141', '7068', '02238',
       '07920', '02138', '01830', '02421', 'K2E1A', '4527-', '07068',
       '00238', 'CO', '01803', '08090', 'ONLINE', 'V6B 1A7', '8020',
       '07660', 'FEE', '08854', '02476', '07656', 'N/A', '00000',
       'M6K 3M1', '01702', '....', '03102', '06902', '01420', '07070',
       'J1H 1', '8022', '99999', 'K1V7P', '01862', '05701', '2138', '2238',
       '06104', '07840', '08054', 'L-2449', '06183', 'V6B 4N4', '1524',
       '05702', '01886', '02494', '06120', '0000000000', 'PO15 5RQ',
       '01230', 'UNKNOWN', '0026', '03261', '02210', '07733', '2451.0',
       '2421.0', '2139.0', '7078.0', '6880', '9027', '07006', 'J1H 1Z3',
       '07751', '04106', '08831', '8112', '8017', 'M2N5J9', '06850',
       '6183', '02452', '01062', '06150', '08053', '8008', 'TEXAS',
       '01063', '08736', '07071', '08002-0343', '02238-2110', '8505',
       'W1G9Q', '8.012', '02050', '8427', 'WASHINGTON', '04101',
       'UNKNOWN 

But `08542` is a valid zip code.

How about the `date_clean`?

In [17]:
print df[df['date_clean'] == datetime.date(year=1990, month=1, day=1)]['filedDate'].unique()

['825178' '831858' '825174' 'Other' 'Credit/Debit Card' 'Cash' 'Check'
 'Electronic Pay System' 'FUND RAISING' nan]
